**Section 5a: Empirical Performance - CBCL Faces**

*Imports*

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.datasets import fetch_olivetti_faces
from collections import defaultdict
from random_matrix import *
from nmf import *
from benchmark import *

---

**Application 1: CBCL Faces Database**

In [5]:
X_faces = fetch_olivetti_faces(shuffle=True).data

*Method Benchmarks*

In [6]:
methods = {
    "MU C": nmf_compress_mu,
    'MU SC': nmf_structured_compress_mu,
    'HALS C': nmf_compress_hals,
    'HALS SC': nmf_structured_compress_hals
}

projection_types = [
    'gaussian',
    # 'srht',
    # 'givens',
    'srft',
    'sparse-jl',
    'count-sketch',
]

stats = benchmark_faces(X_faces,methods,projection_types,r=49,runs=10)
stats

Completed MU C,gaussian
Completed MU C,srft
Completed MU C,sparse-jl
Completed MU C,count-sketch
Completed MU SC,gaussian
Completed MU SC,srft
Completed MU SC,sparse-jl
Completed MU SC,count-sketch
Completed HALS C,gaussian
Completed HALS C,srft
Completed HALS C,sparse-jl
Completed HALS C,count-sketch
Completed HALS SC,gaussian
Completed HALS SC,srft
Completed HALS SC,sparse-jl
Completed HALS SC,count-sketch


errors                                       time            \
projection count-sketch  gaussian sparse-jl      srft count-sketch  gaussian   
algorithm                                                                      
HALS C         0.180025  0.173771  0.175253  0.203729     0.857466  0.870101   
HALS SC        0.149188  0.149196  0.149464  0.149225     0.991161  0.960937   
MU C           0.188077  0.190084  0.185767  0.225832     1.000846  1.015984   
MU SC          0.180837  0.180811  0.180819  0.180833     1.095895  1.102238   

                                
projection sparse-jl      srft  
algorithm                       
HALS C      0.879042  0.876771  
HALS SC     0.979545  2.557963  
MU C        1.019279  1.026006  
MU SC       1.089620  2.686798

In [15]:
stats.round(4)

errors                                    time           \
projection count-sketch gaussian sparse-jl    srft count-sketch gaussian   
algorithm                                                                  
HALS C           0.1800   0.1738    0.1753  0.2037       0.8575   0.8701   
HALS SC          0.1492   0.1492    0.1495  0.1492       0.9912   0.9609   
MU C             0.1881   0.1901    0.1858  0.2258       1.0008   1.0160   
MU SC            0.1808   0.1808    0.1808  0.1808       1.0959   1.1022   

                              
projection sparse-jl    srft  
algorithm                     
HALS C        0.8790  0.8768  
HALS SC       0.9795  2.5580  
MU C          1.0193  1.0260  
MU SC         1.0896  2.6868

**Multiplicative Updates**

In [7]:
# Standard MU Algorithm
X_mu,Y_mu,errors_mu = nmf_mu(X_faces,49,max_iter=100)

# Standard Compressed MU
X_c_mu,Y_c_mu,errors_s_mu = nmf_compress_mu(X_faces,49,max_iter=100,projection_type='count-sketch')

# Structured Compressed MU
X_sc_mu,Y_sc_mu,errors_sc_mu = nmf_structured_compress_mu(X_faces,49,max_iter=100,projection_type='gaussian')

# Errors 
mu_errors_df = pd.DataFrame({'MU':errors_mu,
              'C MU':errors_s_mu,
              'SC MU':errors_sc_mu})

*Plot: Reconstruction Errors*

In [9]:
fig = go.Figure()

for method in mu_errors_df.columns:
    fig.add_trace(
        go.Scatter(
            y = mu_errors_df[method],
            name = method
        )
    )

fig.update_layout(
    title = 'CBCL Reconstruction Error by MU Method (Best Projection)',
    yaxis_title = 'Reconstruction Error',
    xaxis_title = 'Iteration'
)
fig.show()

*Plot: Visual Reconstruction Error by Method*

In [10]:
n_images = 4

fig = make_subplots(
    rows=4, cols=4,
    subplot_titles=(["Original Images"] + [' ']*3 + 
                   ["MU"] + [' ']*3 + 
                   ["Comp. MU (Count-Sketch)"] + [' ']*3 + 
                   ["Struct. Comp. MU (Gaussian)"] + [' ']*3),
    vertical_spacing=0.05,
    horizontal_spacing=0.02
)

matrix_pairs = [
    ("Original", X_faces),
    ("MU", X_mu @ Y_mu),
    ("Compressed MU", X_c_mu @ Y_c_mu),
    ("Structured Compressed MU", X_sc_mu @ Y_sc_mu)
]

for row, (name, Z) in enumerate(matrix_pairs, start=1):
    for col in range(1, n_images + 1):
        z = Z[col-1].reshape(64, 64)
        fig.add_trace(
            go.Heatmap(z=z, colorscale='gray', showscale=False),
            row=row, col=col
        )
  
fig.update_layout(
    title="NMF MU Reconstruction",
    width=800,
    height=600,
    font=dict(size=8), 
    margin=dict(l=10, r=10, b=10, t=40, pad=0), 
    plot_bgcolor='white', 
    paper_bgcolor='white'
)

fig.update_annotations(font_size=10)
fig.update_yaxes(autorange='reversed')
fig.show()

**Hierarchical Least Squares**

In [11]:
# Standard HALS Algorithm
X_hals,Y_hals,errors_hals = nmf_hals(X_faces,49,max_iter=100)

# Standard Compressed HALS
X_c_hals,Y_c_hals,errors_s_hals = nmf_compress_hals(X_faces,49,max_iter=100,projection_type='gaussian')

# Structured Compressed HALS
X_sc_hals,Y_sc_hals,errors_sc_hals = nmf_structured_compress_hals(X_faces,49,max_iter=100,projection_type='count-sketch')

# Errors dataframe
hals_errors_df = pd.DataFrame({'HALS':errors_hals,
              'C HALS':errors_s_hals,
              'SC HALS':errors_sc_hals})

*Plot: Reconstruction Errors*

In [12]:
fig = go.Figure()

for method in hals_errors_df.columns:
    fig.add_trace(
        go.Scatter(
            y = hals_errors_df[method],
            name = method
        )
    )

fig.update_layout(
    title = 'CBCL Reconstruction Error by HALS Method',
    yaxis_title = 'Reconstruction Error',
    xaxis_title = 'Iteration'
)
fig.show()

*Plot: Visual Reconstruction Error by Method*

In [13]:
n_images = 4

fig = make_subplots(
    rows=4, cols=4,
    subplot_titles=(["Original Images"] + [' ']*3 + 
                   ["HALS"] + [' ']*3 + 
                   ["Comp. HALS (Gaussian)"] + [' ']*3 + 
                   ["Struct. Comp. HALS (Count-Sketch)"] + [' ']*3),
    vertical_spacing=0.05,
    horizontal_spacing=0.02
)

matrix_pairs = [
    ("Original", X_faces),
    ("HALS", X_hals @ Y_hals),
    ("Compressed HALS", X_c_hals @ Y_c_hals),
    ("Structured Compressed HALS", X_sc_hals @ Y_sc_hals)
]

for row, (name, Z) in enumerate(matrix_pairs, start=1):
    for col in range(1, n_images + 1):
        z = Z[col-1].reshape(64, 64)
        fig.add_trace(
            go.Heatmap(z=z, colorscale='gray', showscale=False),
            row=row, col=col
        )

fig.update_layout(
    title="NMF HALS Reconstruction",
    width=800,
    height=600,
    font=dict(size=8),  
    margin=dict(l=10, r=10, b=10, t=40, pad=0), 
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.update_annotations(font_size=10)
fig.update_yaxes(autorange='reversed')
fig.show()

---